# Correlacion entre datos

In [15]:
import numpy as np
from sklearn.datasets import load_boston
import pandas as pd

In [16]:
boston = load_boston()
df = pd.DataFrame(boston.data)

In [103]:
df.columns = boston.feature_names
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [30]:
# correlacion entre dos columnas, tener en cuenta que si hay un valor Na
# retornara na, primero se debe limpiar estos valores
np.corrcoef(df['ZN'],df['INDUS'])

array([[ 1.        , -0.53382819],
       [-0.53382819,  1.        ]])

In [38]:
# matriz de correlaciones
corr = round(df.corr(),3)
# le ponemos un degrade para visualizar mas facil
corr.style.background_gradient()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
CRIM,1,-0.2,0.407,-0.056,0.421,-0.219,0.353,-0.38,0.626,0.583,0.29,-0.385,0.456
ZN,-0.2,1,-0.534,-0.043,-0.517,0.312,-0.57,0.664,-0.312,-0.315,-0.392,0.176,-0.413
INDUS,0.407,-0.534,1,0.063,0.764,-0.392,0.645,-0.708,0.595,0.721,0.383,-0.357,0.604
CHAS,-0.056,-0.043,0.063,1,0.091,0.091,0.087,-0.099,-0.007,-0.036,-0.122,0.049,-0.054
NOX,0.421,-0.517,0.764,0.091,1,-0.302,0.731,-0.769,0.611,0.668,0.189,-0.38,0.591
RM,-0.219,0.312,-0.392,0.091,-0.302,1,-0.24,0.205,-0.21,-0.292,-0.356,0.128,-0.614
AGE,0.353,-0.57,0.645,0.087,0.731,-0.24,1,-0.748,0.456,0.506,0.262,-0.274,0.602
DIS,-0.38,0.664,-0.708,-0.099,-0.769,0.205,-0.748,1,-0.495,-0.534,-0.232,0.292,-0.497
RAD,0.626,-0.312,0.595,-0.007,0.611,-0.21,0.456,-0.495,1,0.91,0.465,-0.444,0.489
TAX,0.583,-0.315,0.721,-0.036,0.668,-0.292,0.506,-0.534,0.91,1,0.461,-0.442,0.544


# Análisis de datos extremos

In [43]:
X = df['AGE'].dropna()

In [45]:
X.describe()

count    506.000000
mean      68.574901
std       28.148861
min        2.900000
25%       45.025000
50%       77.500000
75%       94.075000
max      100.000000
Name: AGE, dtype: float64

In [60]:
q1 = np.percentile(X,25)
q3 = np.percentile(X,75)
rango = q3 - q1

In [61]:
umbralSup = q1 + 1.5*rango
umbralInf = q3 - 1.5*rango

In [64]:
umbralInf

20.5

In [65]:
umbralSup

118.6

In [71]:
# porcentaje de muestras fuera del umbral superior
np.mean(X > umbralSup)

0.0

In [72]:
# porcentaje de muestras fuera del umbral inferior
np.mean(X < umbralInf)

0.0691699604743083

In [75]:
from sklearn.covariance import EllipticEnvelope

In [81]:
# modelo para encontrar los valores fuera del rango (outliers)
outliers = EllipticEnvelope(contamination = .01)

In [90]:
var_list = ['AGE','DIS']
x = np.array(df[var_list].dropna())

In [92]:
outliers.fit(x)

EllipticEnvelope(assume_centered=False, contamination=0.01, random_state=None,
                 store_precision=True, support_fraction=None)

In [95]:
pred = outliers.predict(x)

In [97]:
np.where(pred==-1)[0]

array([ 64, 351, 352, 353, 354, 355], dtype=int64)

# Transformacopn de data frame a base de datos relacional

In [106]:
data = [(1,"Pepe","Grillo", 25, 1, "Libreta", 1.2,.4,.8,3,"03-12-2018"),
        (1,"Pepe","Grillo", 25, 2, "Goma", .4,.15,.25,1,"12-2-2018"),
        (1,"Pepe","Grillo", 25, 1, "Libreta", 1.2,.4,.8,2,"03-12-2018"),
        (2,"Juan","Casanova", 33, 2, "Goma", .4,.15,.25,1,"12-2-2018"),
        (2,"Juan","Casanova", 25, 1, "Libreta", 1.2,.4,.8,3,"03-12-2018"),
        (3,"Luis Alberto","Spinetta", 43, 1, "Libreta", 1.2,.4,.8,1,"03-06-2018"),
        (3,"Luis Alberto","Spinetta", 43, 1, "Libreta", 1.2,.4,.8,3,"03-08-2018")] 
labels = ["cli_id", "cli_nombre", "cli_apellido", "cli_edad", "pro_id",
          "pro_name","precio","coste","margen","cantidad","fecha"]

In [109]:
df = pd.DataFrame.from_records(data, columns = labels)
df

,cli_id,cli_nombre,cli_apellido,cli_edad,pro_id,pro_name,precio,coste,margen,cantidad,fecha
0,1,Pepe,Grillo,25,1,Libreta,1.2,0.40,0.80,3,03-12-2018
1,1,Pepe,Grillo,25,2,Goma,0.4,0.15,0.25,1,12-2-2018
2,1,Pepe,Grillo,25,1,Libreta,1.2,0.40,0.80,2,03-12-2018
3,2,Juan,Casanova,33,2,Goma,0.4,0.15,0.25,1,12-2-2018
4,2,Juan,Casanova,25,1,Libreta,1.2,0.40,0.80,3,03-12-2018
5,3,Luis Alberto,Spinetta,43,1,Libreta,1.2,0.40,0.80,1,03-06-2018
6,3,Luis Alberto,Spinetta,43,1,Libreta,1.2,0.40,0.80,3,03-08-2018


In [117]:
clientes = df.drop_duplicates(subset = 'cli_id', keep="first")
clientes

,cli_id,cli_nombre,cli_apellido,cli_edad,pro_id,pro_name,precio,coste,margen,cantidad,fecha
0,1,Pepe,Grillo,25,1,Libreta,1.2,0.40,0.80,3,03-12-2018
3,2,Juan,Casanova,33,2,Goma,0.4,0.15,0.25,1,12-2-2018
5,3,Luis Alberto,Spinetta,43,1,Libreta,1.2,0.40,0.80,1,03-06-2018


In [118]:
clientes = compradores[['cli_id','cli_nombre','cli_apellido','cli_edad']]
clientes

,cli_id,cli_nombre,cli_apellido,cli_edad
0,1,Pepe,Grillo,25
3,2,Juan,Casanova,33
5,3,Luis Alberto,Spinetta,43


In [120]:
productos = df.drop_duplicates(subset= 'pro_id', keep='first')

In [122]:
productos = df[['pro_id','pro_name','precio','coste','margen']]
productos

,pro_id,pro_name,precio,coste,margen
0,1,Libreta,1.2,0.40,0.80
1,2,Goma,0.4,0.15,0.25
2,1,Libreta,1.2,0.40,0.80
3,2,Goma,0.4,0.15,0.25
4,1,Libreta,1.2,0.40,0.80
5,1,Libreta,1.2,0.40,0.80
6,1,Libreta,1.2,0.40,0.80


In [123]:
compras = df[['cli_id','pro_id','fecha','cantidad']]
compras

,cli_id,pro_id,fecha,cantidad
0,1,1,03-12-2018,3
1,1,2,12-2-2018,1
2,1,1,03-12-2018,2
3,2,2,12-2-2018,1
4,2,1,03-12-2018,3
5,3,1,03-06-2018,1
6,3,1,03-08-2018,3
